**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [6]:
# Import the data set
data = pd.read_csv("cleaned_data.csv")

In [7]:
# Define the sigmoid function
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y


**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [8]:
# Create a train/test split
X_train, X_test, y_train, y_test = train_test_split(data[['PAY_1', 'LIMIT_BAL']].values, data['default payment next month'].values,
                                                    test_size=0.2, random_state=24)


______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [10]:
# Fit the logistic regression model on training data
lr.fit(X_train,y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
# Make predictions using `.predict()`
model = lr.predict(X_test)


In [12]:
# Find class probabilities using `.predict_proba()`
model_proba = lr.predict_proba(X_test)


______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [13]:
# Add column of 1s to features
features = np.hstack([np.ones((X_test.shape[0],1)), X_test])


In [14]:
# Get coefficients and intercepts from trained model
coef_inter = np.concatenate([lr.intercept_.reshape(1,1), lr.coef_], axis=1)
coef_inter


array([[-6.57647457e-11,  8.27451187e-11, -6.80876727e-06]])

In [15]:
# Manually calculate predicted probabilities
X_lin = np.dot(coef_inter, np.transpose(features))
model_proba_manual = sigmoid(X_lin)



______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [16]:
# Manually calculate predicted classes
model_manual = model_proba_manual >= 0.5


In [17]:
# Compare to scikit-learn's predicted classes
np.array_equal(model.reshape(1,-1), model_manual)


True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [18]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
roc_auc_score(y_test, model_proba[:,1])


0.627207450280691

In [19]:
# Use manually calculated predicted probabilities to calculate ROC AUC

roc_auc_score(y_test, model_proba_manual.reshape(model_proba_manual.shape[1],))


0.627207450280691